In [10]:
import os
import yaml
import pandas as pd
from metadata import fetch_all_tables, fetch_table_metadata
from col_description import generate_column_descriptions
from dbml_file import generate_dbml_file, publish_dbdocs
import pandas as pd
from sqlalchemy import create_engine, inspect
import openai
import re
from dotenv import load_dotenv

In [11]:
with open("user_config.yaml") as f:
        config = yaml.safe_load(f)


In [12]:
db_uri = config["db_uri"]
schema = config["schema"]
prompt = config['user_prompt']
load_dotenv(config['dotenv_path'])

True

In [13]:
tables = fetch_all_tables(db_uri,schema )

In [14]:
tables

['Artist',
 'Album',
 'Employee',
 'Customer',
 'Invoice',
 'InvoiceLine',
 'Track',
 'Playlist',
 'PlaylistTrack',
 'Genre',
 'MediaType']

In [15]:
df_schema = fetch_table_metadata(db_uri,schema, tables)

In [16]:
df_schema

,Table,Col,Data Type,Key,Index,Description
0,Artist,ArtistId,INTEGER,PK,No,None
1,Artist,Name,VARCHAR(120),,No,None
2,Album,AlbumId,INTEGER,PK,No,None
3,Album,Title,VARCHAR(160),,No,None
4,Album,ArtistId,INTEGER,FK,Yes,None
...,...,...,...,...,...,...
59,PlaylistTrack,TrackId,INTEGER,PK,Yes,None
60,Genre,GenreId,INTEGER,PK,No,None
61,Genre,Name,VARCHAR(120),,No,None
62,MediaType,MediaTypeId,INTEGER,PK,No,None


In [8]:
#Write descriptions to cols using gpt-2.5-turbo
df_schema_updated = generate_column_descriptions(df_schema, 'user_config.yaml')

In [17]:
df_schema_updated

,Table,Col,Data Type,Key,Index,Description
0,Artist,ArtistId,INTEGER,PK,No,ID for artist
1,Artist,Name,VARCHAR(120),,No,Artist name
2,Album,AlbumId,INTEGER,PK,No,ID for album
3,Album,Title,VARCHAR(160),,No,Album title
4,Album,ArtistId,INTEGER,FK,Yes,ID for artist
...,...,...,...,...,...,...
59,PlaylistTrack,TrackId,INTEGER,PK,Yes,ID for track
60,Genre,GenreId,INTEGER,PK,No,ID for genre
61,Genre,Name,VARCHAR(120),,No,Genre name
62,MediaType,MediaTypeId,INTEGER,PK,No,ID for media type


In [18]:
generate_dbml_file(df_schema_updated, config['output_filename'])

✅ DBML file written to: chinhook.dbml


In [19]:
publish_dbdocs('chinhook.dbml', 'chinhookv2', 'paramiyer@gmail.com')

✅ DBML published successfully.

🔗 View your schema: https://dbdocs.io/paramiyer/chinhookv2
